# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

This is a binary classification problem because the target variable we're trying to predict is either 1 or 0, representing whether intervention is needed or not, respectively.

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [3]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = student_data.shape[0]
n_features = student_data.shape[1] - 1
n_passed = student_data['passed'].value_counts()['yes']
n_failed = student_data['passed'].value_counts()['no']
grad_rate = 100 * float(n_passed) / (n_passed + n_failed)
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [4]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [5]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [6]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=0)
print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

perceptron
k-nearest
svc
random forest
gradient boosting classifier
adaboostclassifier

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [118]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn.svm import svc
from sklearn.linear_model import Perceptron
clf = Perceptron(penalty='l1', alpha=.01, random_state=0) # alpha is for regularization
# penalty: None         alpha:        f1score train: 0.256 f1score test: 0.2
# penalty: l1           alpha: 0.01   f1score train: 0.583 f1score test: 0.568 // best for perceptron
# penalty: 'elasticnet' alpha: 0.0001 f1score train: 
# try elasticnet, l1, l2 reg alpha=0.0001

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it

Training Perceptron...
Done!
Training time (secs): 0.001


In [119]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using Perceptron...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.583333333333


In [120]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using Perceptron...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.568421052632


In [125]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant
for size in (100, 200, 300):
    train_predict(clf, X_train[:size], y_train[:size], X_test, y_test)

------------------------------------------
Training set size: 100
Training Perceptron...
Done!
Training time (secs): 0.001
Predicting labels using Perceptron...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.78527607362
Predicting labels using Perceptron...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.781456953642
------------------------------------------
Training set size: 200
Training Perceptron...
Done!
Training time (secs): 0.001
Predicting labels using Perceptron...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.798798798799
Predicting labels using Perceptron...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.766233766234
------------------------------------------
Training set size: 300
Training Perceptron...
Done!
Training time (secs): 0.001
Predicting labels using Perceptron...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.583333333333
Predicting labels using Perceptron...
Done!
Prediction time 

In [266]:
# TODO: Train and predict using two other models

def my_train_predict(clf, X_train, y_train, X_test, y_test, description):
    results = {'description': description}
    
    results['clf'] = clf.__class__.__name__
    
    # Train
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    results['time_train'] = end - start

    # Predict train
    start = time.time()
    y_hat = clf.predict(X_train)
    end = time.time()
    results['f1_score_train'] = f1_score(y_train.values, y_hat, pos_label='yes')
    results['time_predict_train'] = end - start

    # Predict test
    start = time.time()
    y_hat = clf.predict(X_test)
    end = time.time()
    results['f1_score_test'] = f1_score(y_test.values, y_hat, pos_label='yes')
    results['time_predict_test'] = end - start

    return results

def run_clf(clf, results=[]):
    for size in (100, 200, 300):
	results.append(my_train_predict(clf, X_train[:size], y_train[:size], X_test, y_test,"set size: {}".format(size)))


# Checking 6 models intead of 3
results = []

# Re-running first clf, Perceptron, just so it's easier to compare using my own function
clf_perceptron = Perceptron(penalty='l1', alpha=.01, random_state=0)
run_clf(
    clf_perceptron,
    results=results)

# SVC
from sklearn.svm import SVC
clf_svc = SVC(C=1.0, kernel='poly', degree=2, gamma='auto')
run_clf(
    clf_svc,
    results=results)

# K-nearest
from sklearn.neighbors import KNeighborsClassifier as KNC
clf_knc = KNC(n_neighbors=3, weights='uniform')
run_clf(
    clf_knc,
    results=results)

# Random forest
from sklearn.ensemble import RandomForestClassifier as RFC
clf_rfc = RFC(n_estimators=10, max_depth=None, min_samples_split=2)
run_clf(
    clf_rfc,
    results=results)

# GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier as GBC
clf_gbc = GBC(loss='deviance', learning_rate=0.1, n_estimators=100, max_depth=None, min_samples_split=2)
run_clf(
    clf_gbc,
    results=results)

# AdaBoostClassifier with SVC
from sklearn.ensemble import AdaBoostClassifier as ABC
clf_abc = ABC(clf_svc, n_estimators=100, learning_rate=1.0, algorithm='SAMME')
run_clf(
    clf_abc,
    results=results)

pd.DataFrame(results)

,clf,description,f1_score_test,f1_score_train,time_predict_test,time_predict_train,time_train
0,Perceptron,set size: 100,0.781457,0.785276,0.000151,0.000164,0.000789
1,Perceptron,set size: 200,0.766234,0.798799,0.000133,0.000156,0.000954
2,Perceptron,set size: 300,0.568421,0.583333,0.000219,0.000240,0.001615
3,SVC,set size: 100,0.753846,0.924242,0.000540,0.000566,0.001754
4,SVC,set size: 200,0.772727,0.895833,0.000829,0.001886,0.008664
5,SVC,set size: 300,0.776978,0.887892,0.000988,0.003259,0.011225
6,KNeighborsClassifier,set size: 100,0.666667,0.823529,0.000866,0.001003,0.000438
7,KNeighborsClassifier,set size: 200,0.666667,0.861210,0.001261,0.002439,0.000473
8,KNeighborsClassifier,set size: 300,0.721805,0.886878,0.002206,0.004818,0.000614
9,RandomForestClassifier,set size: 100,0.730159,0.992126,0.000771,0.000905,0.023050


## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score? .808

In [ ]:
# TODO: Fine-tune your model and report the best F1 score

In [279]:
# Fine-tuning three best methods from above, just be sure, since they were pretty close

from sklearn.grid_search import GridSearchCV as GSCV

# Must convert target values to binary, cuz grid search doesn't work with string targets, for some reason
y_train_binary = y_train.apply(lambda v: v=='yes')
y_test_binary = y_test.apply(lambda v: v=='yes')

# Fine-tuning SVC
best_f1_score = None
best_params = None
for i in range(25):
    #print "iteration {}".format(i)
    gs_clf = GSCV(
	clf_svc,
	{'kernel':['poly'], 'C':[.95, 1, 1.05], 'degree':[1], 'gamma': ['auto']},
	scoring='f1',
	cv=10,
	n_jobs=4)
    gs_clf.fit(X_train, y_train_binary)
    _f1_score = f1_score(y_test_binary, gs_clf.predict(X_test))
    if best_f1_score is None or best_f1_score < _f1_score:
	best_f1_score = _f1_score
	best_params = {
	    'C': gs_clf.best_estimator_.C,
	    'degree': gs_clf.best_estimator_.degree,
	    'kernel': gs_clf.best_estimator_.kernel,
        'gamma' : gs_clf.best_estimator_.gamma}
    
print best_f1_score, best_params

#{'kernel':('linear', 'rbf', 'poly'), 'C':[.1, 1, 10], 'degree':[1,2,3]},
#0.794520547945 {'kernel': 'poly', 'C': 1, 'degree': 1}

#{'kernel':['poly'], 'C':[0.9, 1, 1.1], 'degree':[1,2]},
#0.794520547945 {'kernel': 'poly', 'C': 1, 'degree': 1}

# {'kernel':('linear', 'rbf', 'poly'), 'C':[.1, 1, 10], 'degree':[1,2,3], 'gamma': ['auto',1,5,10]}
# 0.766233766234 {'kernel': 'rbf', 'C': 0.1, 'degree': 1}


# {'kernel':['poly', 'rbf', 'linear'], 'C':[.1, 1, 10], 'degree':[1,2,3], 'gamma': ['auto', 1, .1, .01]}
# 0.794520547945 {'kernel': 'poly', 'C': 1, 'gamma': 'auto', 'degree': 1}

# {'kernel':['poly', 'rbf'], 'C':[.95, 1, 1.05], 'degree':[1], 'gamma': ['auto', .001, .0001]}
# 0.789115646259 {'kernel': 'poly', 'C': 0.95, 'gamma': 'auto', 'degree': 1}

# {'kernel':['rbf'], 'C':[.95, 1, 1.05], 'degree':[1], 'gamma': [.001, .0001, .00001, .000001, .0000001]}
# 0.766233766234 {'kernel': 'rbf', 'C': 0.95, 'gamma': 0.0001, 'degree': 1}

# {'kernel':['poly'], 'C':[.95, 1, 1.05], 'degree':[1], 'gamma': ['auto']},
# 0.794520547945 {'kernel': 'poly', 'C': 1, 'gamma': 'auto', 'degree': 1}

0.794520547945 {'kernel': 'poly', 'C': 1, 'gamma': 'auto', 'degree': 1}


In [197]:
# Fine-tuning random forest

best_f1_score = None
best_params = None
for i in range(50):
    if i < 3 or i % 10 == 0:
	print "iteration {}".format(i)
	start = time.time()
    gs_clf = GSCV(
	clf_rfc,
	{'n_estimators': [14, 15, 16, 18], 'max_depth': [4, 5, 6, 7, 8], 'min_samples_split': [2, 3, 4, 5]},
	scoring='f1',
	cv=5,
	n_jobs=4)
    gs_clf.fit(X_train, y_train_binary)
    _f1_score = f1_score(y_test_binary, gs_clf.predict(X_test))
    if best_f1_score is None or best_f1_score < _f1_score:
	best_f1_score = _f1_score
	best_params = {
	    'n_estimators': gs_clf.best_estimator_.n_estimators,
	    'max_depth': gs_clf.best_estimator_.max_depth,
	    'min_samples_split': gs_clf.best_estimator_.min_samples_split}
    if i < 3:
	end = time.time()    
	print "Each iteration time(secs): {:.3f}".format(end - start)
	
print best_f1_score, best_params

# i in range(10)
# {'n_estimators': [10, 15, 20], 'max_depth': [None, 2, 4, 8, 16], 'min_samples_split': [2, 4, 8, 16]}
# 0.802721088435 {'n_estimators': 15, 'min_samples_split': 4, 'max_depth': 4}


0.808219178082 {'n_estimators': 15, 'min_samples_split': 3, 'max_depth': 8}
iteration 40
iteration 30
iteration 20
iteration 10
Each iteration time(secs): 3.553
Each iteration time(secs): 3.571
iteration 2
Each iteration time(secs): 3.533
iteration 1
iteration 0


In [283]:
# Fine tuning AdaBoostClassifier with SVC
# Fine-tuning SVC

# Create svc with optimized params
svc2 = SVC(C=1.0, kernel='poly', degree=1, gamma='auto', probability=True)
abc2 = ABC(svc2, algorithm='SAMME')


best_f1_score = None
best_params = None
for i in range(5):
    #print "iteration {}".format(i)
    gs_clf = GSCV(
        abc2,
        {'n_estimators': [5, 50, 100], 'learning_rate': [.0001, .001, .01]},
        scoring='f1',
        cv=5,
        n_jobs=4)
    gs_clf.fit(X_train, y_train_binary)
    _f1_score = f1_score(y_test_binary, gs_clf.predict(X_test))
    if best_f1_score is None or best_f1_score < _f1_score:
        best_f1_score = _f1_score
        best_params = {
            'n_estimators': gs_clf.best_estimator_.n_estimators,
            'learning_rate': gs_clf.best_estimator_.learning_rate}
    
print best_f1_score, best_params

# {'n_estimators': [5, 10, 50, 100], 'learning_rate': [.001, .1, 1.0]}
# 0.766233766234 {'n_estimators': 5, 'learning_rate': 0.001}

# {'n_estimators': [5, 50, 100], 'learning_rate': [.0001, .001, .01]},
# 0.766233766234 {'n_estimators': 5, 'learning_rate': 0.0001}

# Gridsearchcv just returns the same results no matter what params i put in

0.766233766234 {'n_estimators': 5, 'learning_rate': 0.0001}
